In [1]:
__author__ = "Olivier Witteman"
__email__ = "olivier@2001.net"

from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
import numpy as np
import time
import scipy.stats as sps
from scitime import Estimator
import time_complexity as tcom

In [2]:
# General parameters
n = 5

# Dataset parameters
features = 3000
set_size = int(1e4)

X, y = make_regression(n_samples=set_size, n_features=features, random_state=0)

# Random forest parameters
max_trees = 100
tries_tree = set_size

# Neural network parameters
output_nodes = 1
iterations = 30
hidden_layers = (8, 100, 100, 100, 100, 1)
batch_size = 200

In [3]:
estimator = Estimator(meta_algo='RF', verbose=2)
rfr = RandomForestRegressor(n_estimators=max_trees)

# Estimation by scitime
estimation, lower_bound, upper_bound = estimator.time(rfr, X, y=y)

tlst_rf = []

# for _ in range(n):
#     t0 = time.time()
#     rfr.fit(X, y)
#     tlst_rf.append(time.time()-t0)
# 
#     print('\r{!s}'.format(sps.describe(tlst_rf)), end='')

scitime.estimate.Estimator:INFO:Training your model for these params: {'total_memory': [8589934592], 'available_memory': [2733043712], 'num_cpu': [4], 'num_rows': [10000], 'num_features': [3000], 'n_estimators': [100], 'max_depth': [None], 'min_samples_split': [2], 'min_samples_leaf': [1], 'min_weight_fraction_leaf': [0.0], 'max_features': ['auto'], 'criterion': ['mse'], 'max_leaf_nodes': [None], 'min_impurity_decrease': [0.0], 'min_impurity_split': [None], 'bootstrap': [True], 'oob_score': [False], 'n_jobs': [None]}
scitime.estimate.Estimator:INFO:Transforming dataset for semi dummy features
scitime.estimate.Estimator:WARNING:Parameters ['max_samples', 'random_state', 'warm_start', 'ccp_alpha', 'verbose'] will not be accounted for
scitime.estimate.Estimator:INFO:Training your RandomForestRegressor model should take ~ 7 minutes
                and 11 seconds
scitime.estimate.Estimator:INFO:The 95.0% prediction interval is [0 seconds, 45 minutes
                and 22 seconds]


In [4]:
trees = len(rfr.estimators_)
t_b = tcom.big_o_rfr(n_base=np.array([estimation, lower_bound, upper_bound]), n_tree=trees, m_try=tries_tree,
                     n=min(set_size, int(2e4)), inv=True)

nn_est = tcom.big_o_nn(n_base=t_b, m=features, o=output_nodes, i=iterations, nodes=hidden_layers, t=set_size, inv=False)
print('Neural network training time estimate (95% confidence interval):\n\nEstimate:    {!s}\nLower bound: '
      '{!s}\nUpper bound: {!s}'.format(nn_est[0], nn_est[1], nn_est[2]))

TypeError: 'int' object is not subscriptable

In [ ]:
nn = MLPRegressor(hidden_layer_sizes=hidden_layers, max_iter=iterations, batch_size=batch_size)

tlst_nn = []

for _ in range(n):
    t0 = time.time()
    nn.fit(X, y)
    tlst_nn.append(time.time()-t0)

    print('\r{!s}'.format(sps.describe(tlst_nn)), end='')